# Detección de contenido explícito, sugerente y violento con Amazon Rekognition

In [ ]:
import boto3
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
bucketName = "{REPLACE_YOUR_BUCKET}"
imageName = "media/yoga_swimwear_resized.jpg"

In [ ]:
s3.upload_file(
    Filename="resources/yoga_swimwear_resized.jpg",
    Bucket=bucketName,
    Key=imageName,
)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

In [ ]:
detectModerationLabelsResponse = rekognition.detect_moderation_labels(
   Image={
       'S3Object': {
           'Bucket': bucketName,
           'Name': imageName,
       }
   }
)

In [ ]:
display(detectModerationLabelsResponse)

In [ ]:
for label in detectModerationLabelsResponse["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))

# Moderacion de contenido en videos

In [ ]:
bucketName = "temporal-bucket-max"
videoName = "media/moderation.mov"

In [ ]:
s3.upload_file(
    Filename="resources/moderation.mov",
    Bucket=bucketName,
    Key=videoName,
)

In [ ]:
s3VideoUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': videoName})

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))

In [ ]:
startModerationLabelDetection = rekognition.start_content_moderation(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

moderationJobId = startModerationLabelDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

In [ ]:
getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP'
)

while(getContentModeration['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP')
    
display(getContentModeration['JobStatus'])

In [ ]:
theObjects = {}

strDetail = "Moderation labels in video<br>=======================================<br>"
strOverall = "Moderation labels in the overall video:<br>=======================================<br>"

for obj in getContentModeration['ModerationLabels']:
    ts = obj ["Timestamp"]
    cconfidence = obj['ModerationLabel']["Confidence"]
    oname = obj['ModerationLabel']["Name"]
    strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence,2))
    if oname in theObjects:
        cojb = theObjects[oname]
        theObjects[oname] = {"Name" : oname, "Count": 1+cojb["Count"]}
    else:
        theObjects[oname] = {"Name" : oname, "Count": 1}

for theObject in theObjects:
    strOverall = strOverall + "Name: {}, Count: {}<br>".format(theObject, theObjects[theObject]["Count"])

display(HTML(strOverall))
display(HTML(strDetail))

In [ ]:
listui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(strDetail)
display(HTML(listui))